# OBTENCION DE LA INFORMACION MEDIANTE WEB SCRAPING 

### Importamos galerias 

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#pip install beautifulsoup4

In [3]:
# URL de la página que contiene el enlace al archivo que deseas descargar
url_pagina = 'https://data.buenosaires.gob.ar/dataset/victimas-siniestros-viales'

# Realizar la solicitud GET a la página web
response = requests.get(url_pagina)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Analizar el contenido HTML de la página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar el segundo contenedor con la clase 'pkg-container'
    contenedor = soup.find('div', {'id': 'pkg-main-info'}) \
                    .find_all('div', {'class': 'pkg-container'})[1]

    # Buscar el enlace dentro de 'pkg-actions'
    pkg_actions = contenedor.find('div', {'class': 'pkg-actions'})
    enlace_archivo = pkg_actions.find('a', {'class': 'btn'})

    if enlace_archivo:
        # Obtener el valor del atributo href
        url_descarga = enlace_archivo.get('href')

        print('Enlace del archivo:', url_descarga)
    else:
        print('No se encontró el enlace al archivo en el segundo contenedor.')

else:
    print('Error al realizar la solicitud HTTP. Código de estado:', response.status_code)

Enlace del archivo: https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/victimas-siniestros-viales/homicidios.xlsx


### Descargamos el archivo 

In [7]:
response = requests.get(url_descarga)

# Verifica si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Guarda el contenido del archivo en un archivo local
    with open('Dataframes/Hechos.xlsx', 'wb') as f:
        f.write(response.content)

    print('Archivo descargado exitosamente.')
else:
    print(f'Error al descargar el archivo. Código de estado: {response.status_code}')

Archivo descargado exitosamente.


### Pasamos de Excel a CSV 

In [3]:
# pip install openpyxl
# Especifica la ruta de tu archivo Excel
archivo_excel = 'Dataframes/Hechos.xlsx'

# Lee todas las hojas del archivo Excel
hojas_excel = pd.read_excel(archivo_excel, sheet_name=None)

# Especifica la carpeta donde deseas guardar los archivos CSV
carpeta_csv = 'Dataframes/'

# Itera sobre cada hoja y guarda el DataFrame en un archivo CSV
for nombre_hoja, df in hojas_excel.items():
    # Genera el nombre del archivo CSV para cada hoja
    archivo_csv = f'{carpeta_csv}{nombre_hoja}.csv'

    # Guarda el DataFrame en formato CSV
    df.to_csv(archivo_csv, index=False)

    print(f'Archivo CSV "{archivo_csv}" creado exitosamente.')


Archivo CSV "Dataframes/HECHOS.csv" creado exitosamente.
Archivo CSV "Dataframes/DICCIONARIO_HECHOS.csv" creado exitosamente.
Archivo CSV "Dataframes/VICTIMAS.csv" creado exitosamente.
Archivo CSV "Dataframes/DICCIONARIO_VICTIMAS.csv" creado exitosamente.
Archivo CSV "Dataframes/clas.csv" creado exitosamente.


# ETL en base a EDA 

### Cargamos archivos csv en data frames 

In [4]:
hechos = 'Dataframes/HECHOS.csv'

hechos_victimas= 'Dataframes/VICTIMAS.csv'
df_hechos_original = pd.read_csv(hechos, encoding='latin-1')
df_hechos_victimas_original = pd.read_csv(hechos_victimas, encoding='latin-1')

### Visualizamos el data frame hechos 

In [5]:
df_hechos_original.head(10)

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,DirecciÃ³n Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÃA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃâA, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
5,2016-0008,1,2016-01-24,2016,1,24,18:30:00,18,AV 27 DE FEBRERO Y AV ESCALADA,AVENIDA,...,NaN,ESCALADA AV.,27 DE FEBRERO AV. y ESCALADA AV.,8,Point (101721.59002217 93844.25656649),-58.44451316,-34.68475866,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
6,2016-0009,1,2016-01-24,2016,1,24,19:10:00,19,NOGOYA Y JOAQUIN V. GONZALES,CALLE,...,NaN,"GONZALEZ, JOAQUIN V.","NOGOYA y GONZALEZ, JOAQUIN V.",11,Point (96545.87592078 102330.67262199),-58.50095869,-34.60825440,MOTO-AUTO,MOTO,AUTO
7,2016-0010,1,2016-01-29,2016,1,29,15:20:00,15,AV GENERAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,MOTO-AUTO,MOTO,AUTO
8,2016-0012,1,2016-02-08,2016,2,8,01:20:00,1,AV BELGRANO Y BERNARDO DE IRIGOYEN,AVENIDA,...,NaN,"IRIGOYEN, BERNARDO DE","BELGRANO AV. e IRIGOYEN, BERNARDO DE",1,Point (107595.35084333 101797.50052813),-58.38048577,-34.61303893,MOTO-CARGAS,MOTO,CARGAS
9,2016-0013,1,2016-02-10,2016,2,10,11:30:00,11,AV ENTRE RIOS 1366,AVENIDA,...,1366.0,NaN,ENTRE RIOS AV. 1366,1,Point (106616.41069662 100496.44662323),-58.39114932,-34.62477387,PEATON-AUTO,PEATON,AUTO


### Columnas del Data Frame  

In [6]:
df_hechos_original.columns 

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce',
       'DirecciÃ³n Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

### Borramos columnas no necesarias 

In [7]:
df_hechos_modificado = df_hechos_original.drop(['AAAA', 'MM', 'DD', 'HORA' , 'XY (CABA)','PARTICIPANTES'],axis=1 )

### Cambiamos valores Nan por 0 

In [8]:
df_hechos_modificado['N_VICTIMAS'] = df_hechos_modificado['N_VICTIMAS'].fillna(0)
df_hechos_modificado['COMUNA'] = df_hechos_modificado['COMUNA'].fillna(0)

### Pasamos los valores 1.0 2.0 etc a numeros entero 

In [9]:
df_hechos_modificado['N_VICTIMAS'] = df_hechos_modificado['N_VICTIMAS'].map(int)
df_hechos_modificado['COMUNA'] = df_hechos_modificado['COMUNA'].map(int)

### Borramos las filas vacias 

In [10]:
df_hechos_modificado = df_hechos_modificado.dropna(subset=['ID'])

### Visualizamos 


In [11]:
df_hechos_modificado.head(5)

,ID,N_VICTIMAS,FECHA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,DirecciÃ³n Normalizada,COMUNA,pos x,pos y,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,-58.47533969,-34.68757022,MOTO,AUTO
1,2016-0002,1,2016-01-02,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,-58.50877521,-34.66977709,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,2034.0,NaN,ENTRE RIOS AV. 2034,1,-58.39040293,-34.63189362,MOTO,AUTO
3,2016-0004,1,2016-01-10,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,-58.46503904,-34.68092974,MOTO,SD
4,2016-0005,1,2016-01-21,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÃA,AVENIDA,SAN JUAN AV.,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃâA, LUIS, PRES.",1,-58.38718297,-34.62246630,MOTO,PASAJEROS


### Importamos en formato csv 

In [12]:
df_hechos_modificado.to_csv('Dataframes/hechos_clean.csv', index=False)

### Cambiamos el nombre de la columna ID_hecho del data frame victimas por ID  

In [13]:
df_hechos_victimas_modificado = df_hechos_victimas_original.rename(columns={'ID_hecho': 'ID'})

### Borramos columnas que no utilizaremos en el data frame victimas 

In [14]:
df_hechos_victimas_modificado_1 = df_hechos_victimas_modificado.drop([ 'AAAA', 'MM', 'DD'],axis=1 )

### Visualizamos 

In [15]:
df_hechos_victimas_modificado_1.head(5)

,ID,FECHA,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


### Exportamos en formato .csv 

In [21]:
df_hechos_victimas_modificado_1.to_csv('Dataframes/victimas_clean.csv', index=False )

### Hacemos un merge en base a la columna ID de los data frame hechos clean  y victimas clean 

In [16]:
df_merged = pd.merge(df_hechos_modificado, df_hechos_victimas_modificado_1, on='ID', how='inner')

### Visualizamos

In [17]:
df_merged.head(5)

,ID,N_VICTIMAS,FECHA_x,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,DirecciÃ³n Normalizada,...,pos x,pos y,VICTIMA_x,ACUSADO,FECHA_y,ROL,VICTIMA_y,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",...,-58.47533969,-34.68757022,MOTO,AUTO,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,1,2016-01-02,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",...,-58.50877521,-34.66977709,AUTO,PASAJEROS,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,1,2016-01-03,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,2034.0,NaN,ENTRE RIOS AV. 2034,...,-58.39040293,-34.63189362,MOTO,AUTO,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,1,2016-01-10,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",...,-58.46503904,-34.68092974,MOTO,SD,2016-01-10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,1,2016-01-21,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÃA,AVENIDA,SAN JUAN AV.,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃâA, LUIS, PRES.",...,-58.38718297,-34.62246630,MOTO,PASAJEROS,2016-01-21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [24]:
df_merged.columns

Index(['ID', 'N_VICTIMAS', 'FECHA_x', 'HH', 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE',
       'Calle', 'Altura', 'Cruce', 'DirecciÃ³n Normalizada', 'COMUNA', 'pos x',
       'pos y', 'VICTIMA_x', 'ACUSADO', 'FECHA_y', 'ROL', 'VICTIMA_y', 'SEXO',
       'EDAD', 'FECHA_FALLECIMIENTO'],
      dtype='object')

### Elimino columnas duplicadas

In [18]:
df_merged_clean = df_merged.drop(['FECHA_y', 'VICTIMA_y' ], axis = 1)

### Exportamos en formato csv 

In [19]:
df_merged_clean.to_csv('Dataframes/merged_clean.csv', index=False)

## PREPARACION DATA FRAME FINAL 

#### Cargamos el data frame 

In [20]:
final='Dataframes/merged_clean.csv'
dffinal = pd.read_csv(final, encoding='latin-1')

#### Agregamos la copia de dos columnas que nos van a ayudar para la creacion del dash board 

In [21]:
dffinal['COMUNA-copia'] = dffinal['COMUNA'].copy()

In [22]:
dffinal['EDAD-copia'] = dffinal['EDAD'].copy()

In [23]:
dffinal.head(5)

,ID,N_VICTIMAS,FECHA_x,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,DirecciÃÂ³n Normalizada,...,pos x,pos y,VICTIMA_x,ACUSADO,ROL,SEXO,EDAD,FECHA_FALLECIMIENTO,COMUNA-copia,EDAD-copia
0,2016-0001,1,2016-01-01,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",...,-58.47533969,-34.68757022,MOTO,AUTO,CONDUCTOR,MASCULINO,19,2016-01-01 00:00:00,8,19
1,2016-0002,1,2016-01-02,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",...,-58.50877521,-34.66977709,AUTO,PASAJEROS,CONDUCTOR,MASCULINO,70,2016-01-02 00:00:00,9,70
2,2016-0003,1,2016-01-03,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,2034.0,NaN,ENTRE RIOS AV. 2034,...,-58.39040293,-34.63189362,MOTO,AUTO,CONDUCTOR,MASCULINO,30,2016-01-03 00:00:00,1,30
3,2016-0004,1,2016-01-10,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",...,-58.46503904,-34.68092974,MOTO,SD,CONDUCTOR,MASCULINO,18,SD,8,18
4,2016-0005,1,2016-01-21,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÃÂA,AVENIDA,SAN JUAN AV.,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃÂÃ¢ÂÂA, LUIS, PRES.",...,-58.38718297,-34.62246630,MOTO,PASAJEROS,CONDUCTOR,MASCULINO,29,2016-02-01 00:00:00,1,29


#### Remplazamos Valores erroneos en VICTIMA_x y en EDAD-copia

In [24]:
dffinal['VICTIMA_x'] = dffinal['VICTIMA_x'].replace(['PEATON MOTO'], 'SD')

In [25]:
dffinal['EDAD-copia'] = dffinal['EDAD-copia'].replace(['OBJETO FIJO'], '')

In [26]:
dffinal.dtypes

ID                           object
N_VICTIMAS                    int64
FECHA_x                      object
HH                           object
LUGAR_DEL_HECHO              object
TIPO_DE_CALLE                object
Calle                        object
Altura                      float64
Cruce                        object
DirecciÃÂ³n Normalizada     object
COMUNA                        int64
pos x                        object
pos y                        object
VICTIMA_x                    object
ACUSADO                      object
ROL                          object
SEXO                         object
EDAD                         object
FECHA_FALLECIMIENTO          object
COMUNA-copia                  int64
EDAD-copia                   object
dtype: object

In [27]:
dffinal.to_csv('DataFrameFinal.csv', index=False)